In [3]:
import pandas as pd
import csv

In [4]:
df = pd.read_csv('train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df = df.dropna()

In [8]:
X = df.drop('label', axis=1)
y = df['label']

In [9]:
X.shape

(18285, 4)

In [10]:
import tensorflow as tf

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense

In [12]:
vocab_size = 5000

In [13]:
messages = X.copy()

In [14]:
messages.reset_index(inplace = True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [18]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [19]:
onehot_repr = [one_hot(words, vocab_size) for words in corpus]

In [20]:
onehot_repr

[[2451, 458, 2891, 3938, 4809, 2730, 952, 2420, 966, 1531],
 [4009, 2173, 1567, 1405, 1030, 1384, 1597],
 [1668, 3685, 4670, 1961],
 [4704, 397, 3532, 1988, 856, 2769],
 [2561, 1030, 2632, 4417, 3329, 4842, 1030, 870, 2501, 3469],
 [3469,
  2728,
  3048,
  4543,
  1127,
  14,
  888,
  4308,
  2193,
  1458,
  4079,
  92,
  3750,
  4237,
  1597],
 [4883, 3516, 3470, 1833, 4923, 1538, 4750, 1918, 4463, 4840, 921],
 [1201, 952, 2592, 4002, 1416, 2779, 14, 1868, 4463, 4840, 921],
 [1215, 2105, 1933, 795, 4505, 1097, 3870, 960, 14, 3925],
 [3993, 600, 2628, 1373, 1963, 4694, 1992, 1314],
 [359, 229, 4526, 1479, 4940, 1853, 2843, 3801, 1791, 2434, 50],
 [1988, 41, 4809, 1097, 14, 1416],
 [3208, 3552, 4271, 2242, 2062, 538, 4319, 255, 3498],
 [988, 4900, 2248, 3766, 3720, 2825, 954, 4463, 4840, 921],
 [3932, 4842, 823, 4162, 1582, 4463, 4840, 921],
 [813, 3559, 1065, 367, 4837, 3057, 2468, 8, 1987, 2433],
 [1143, 4588, 2173],
 [612, 3333, 2159, 2997, 14, 4057, 935, 1597],
 [1786, 3323, 1567, 9

In [21]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)

In [22]:
vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer  = 'adam', metrics = ['accuracy'])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [24]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [25]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [28]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 20, batch_size = 64)

Epoch 1/20
192/192 [==============================] - 9s 46ms/step - loss: 0.0814 - accuracy: 0.9714 - val_loss: 0.2815 - val_accuracy: 0.9100
Epoch 2/20
192/192 [==============================] - 11s 57ms/step - loss: 0.0507 - accuracy: 0.9833 - val_loss: 0.2993 - val_accuracy: 0.9115
Epoch 3/20
192/192 [==============================] - 8s 41ms/step - loss: 0.0313 - accuracy: 0.9896 - val_loss: 0.3339 - val_accuracy: 0.9065
Epoch 4/20
192/192 [==============================] - 11s 58ms/step - loss: 0.0181 - accuracy: 0.9944 - val_loss: 0.4434 - val_accuracy: 0.9114
Epoch 5/20
192/192 [==============================] - 9s 48ms/step - loss: 0.0121 - accuracy: 0.9962 - val_loss: 0.4455 - val_accuracy: 0.9140
Epoch 6/20
192/192 [==============================] - 10s 50ms/step - loss: 0.0083 - accuracy: 0.9975 - val_loss: 0.5227 - val_accuracy: 0.9110
Epoch 7/20
192/192 [==============================] - 11s 58ms/step - loss: 0.0043 - accuracy: 0.9989 - val_loss: 0.6419 - val_accuracy: 0.

In [29]:
y_pred = model.predict(X_test)

189/189 [==============================] - 2s 7ms/step


In [32]:
y_pred = np.where(y_pred > 0.5, 1.0, 0)

In [35]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [34]:
confusion_matrix(y_test, y_pred)

array([[3125,  294],
       [ 273, 2343]])

In [36]:
accuracy_score(y_test, y_pred)

0.9060480530240265

In [38]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3419
           1       0.89      0.90      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.91      0.91      0.91      6035

